In [1]:
import numpy as np

## Application Instance: Ply Composite Optimization 

This application class concerns the computer-aided design of composite materials for high performance aircraft structures. Composite materials boast excellent specific strength and stiffness properties (strength and stiffness per unit weight) hence their popularity in the aerospace industry. Composite materials are often manufactured using automated fiber placement (AFP) machines that lay-up a series of adjacent unidirectional tapes of polymer pre-impregnated fibers (e.g., carbon-epoxy, boron-epoxy, carbon-PEEK) to form a lamina known as a *ply*. Plies are then stacked on top of one another to form a laminate. The order in which these plies are stacked with respect to their lay-up orientation-*the stacking sequence*-is a critical design degree of freedom. This is because the strength and stiffness properties of a ply may vary significantly in two mutually orthogonal directions within the plane of that ply. The figure below depicts a ply composite design for an airplane wing which is adapted from [1]. Each layer of this wing design corresponds to a distinct orientation, typically $\theta \in \{0^\circ, 90^\circ, \pm 45^\circ\}$. There is also a choice to omit a ply from a given layer within a given panel. Each panel corresponds to a possible choice for a stacking sequence. We aim to generate a library of valid stacking sequences that we can use in a post-processing step that optimizes the entire wing design. There are two versions of this problem, one which includes the absense of a ply at a given layer and one which excludes the absense of a ply.

### Problem Version 1: Disallow Absent Ply Layers

Let $\boldsymbol \theta \in \{0^\circ, 90^\circ, \pm 45^\circ \}^N$ be the design vector. Here $N = n_1 + n_2 + n_3 + n_4$ are given to us by a preprocessing step that determines the so-called *ply gauges* $n_j$ where $n_1$, $n_2$, $n_3$, and $n_4$ are the numbers of $0^\circ$, $45^\circ$, $90^\circ$, and $-45^\circ$ oriented plies, respectively. The core computational challenge for this application instance is then to minimize the following cost function subject to a variety of design rules (constraints),

$C (\boldsymbol \theta) = \sum_{j=1}^4 \left|\sum_{i=1}^N \delta_{\boldsymbol \theta_i, \theta_j} - n_j \right|$,

where $\delta_{\boldsymbol \theta_i, \theta_j} = 1$ if the $i$-th entry of the design $\boldsymbol \theta_i = \theta_j$ where $\theta_j \in \{0^\circ, 90^\circ, \pm 45^\circ \}$, $\delta_{\boldsymbol \theta_i, \theta_j} = 0$ otherwise. More specifically, we are interested in finding a set of solutions $\{ \boldsymbol \theta^{(k)} \}$ such that $C(\boldsymbol \theta^{(k)}) = 0$ subject to the following constraints:

**Symmetry:** The composite part is symmetric about the mid-plane.

**Balance:** The composite part consist of equal $+45^\circ$ and $-45^\circ$ orientations.

**Minimum Percentage:** A minimum percentage of all fibre orientations are used in the composite part, typically $10\%$.

**Damage Tolerance:** A $+45^\circ$ or $-45^\circ$ layer is placed on the outermost part of the composite part.

**Contiguity:** The maximum grouping of consecutive orientations of the same angle is limited, typically no more than $2$ consecutive repeated angles.

$\mathbf{\pm 45^\circ}$ **Rule:** All $\pm 45^\circ$s must be followed by $\mp 45^\circ$ no matter how many layer separate them.

**Disorientation:** The difference in fibre orientation between two adjacent layers *may* be limited to $45^\circ$, note that in this context we consider $90^\circ$ to be $45^\circ$ away from $\pm 45^\circ$.


### Problem Version 2: Allow for Absent Ply Layers

In this version, we now have a design vector $\boldsymbol \theta \in \{\emptyset,0^\circ, 90^\circ, \pm 45^\circ \}^D$
where $\emptyset$ denotes the absense of a ply and $D$ is the total number of ply layers. Note that in this case we have that $N = n_1 + n_2 + n_3 + n_4 \leq D$, typically we will have $N < D$. The cost function and the constraints are the same as above. However, enforcing the above constraints become significantly more difficult given the constraints become non-local and thereby harder to treat algorithmically. The cost function for this version is only slightly different to account for presence of $\emptyset$,

$C (\boldsymbol \theta) = \sum_{j=1}^4 \left|\sum_{i=1}^D \delta_{\boldsymbol \theta_i, \theta_j} - n_j \right|$.

Note that treating design rules (incorporating constraints) for version 2 problems is significantly harder than for version 1 problems due to the non-locality of the constraints.
 
### In this notebook,

we provide code for generating random problem instances of varying complexity for benchmarking purposes. We also provide suggestions for how to employ a branch-and-bound algorithm to solve this combinatorial optimization problem. The ultimate goal is to develop the $\textbf{cost}$ and $\textbf{branch}$ oracles used in quantum branch-and-bound algorithms described in [arXiv:1906.10375](https://arxiv.org/abs/1906.10375), [arXiv:2210.03210](https://arxiv.org/abs/2210.03210).

Let's label $\emptyset \to 0$, $0^\circ \to 1$,  $45^\circ \to 2$, $-45^\circ \to 3$, and $90^\circ \to 4$. We first consider the simpler case of version 1 of this problem. First let's write down a Python function that implements the cost function classically.

In [2]:
def cost(theta, n):
    """
        Implement cost for version 1 problem
        
        theta := partial design vector
        n := ply gauges n = (n1, n2, n3, n4)
    """
    num_of_angles = np.zeros(4, dtype=int)
    for t in theta:
        num_of_angles[t-1] += 1 # theta values range from 1 to 4 w/o empty ply option, 
                                # t -> t-1 is an angle to array index conversion
    
    return sum(abs(num_of_angles-n))

In [3]:
# Example evaluation of cost
n = [3, 5, 2, 8]
theta = np.random.randint(low=1, high=4, size=10)
cost(theta,n)

8

In [27]:
def cost2(theta, n):
    """
        Implement cost for version 2 problem
        
        theta := partial design vector
        n := ply gauges n = (n1, n2, n3, n4)
    """
    num_of_angles = np.zeros(4, dtype=int)
    for t in theta:
        if t > 0: # Don't count empty ply layers
            
            num_of_angles[t-1] += 1 # theta values range from 1 to 4 w/o empty ply option, 
                                    # t -> t-1 is an angle to array index conversion
    
    return sum(abs(num_of_angles-n))

A nice feature of this application instance is that we can generate challenge problems easily. The most common choice for ply gauge distribution is *quasi-isotropic* $(n_{0}, n_{45} + n_{-45},  n_{90})/N =  (25\%, 50\%, 25\%)$. Other possibilities are: $(30\%, 60\%, 10\%), (40\%, 50\%, 10\%), (50\%, 40\%, 10\%), (20\%, 60\%, 20\%), (30\%, 40\%, 30\%)$. When generating version 1 problems, it is sufficient to select a ply gauge distribution and the total number of plies $N$. When generating version 2 problems, we must also select a value for $D$.

In order to generate a random problem, we first choose our ply gauges according to a distribution, say $(25\%, 50\%, 25\%)$. Choose a value for total number of plies $N$, then compute $n_j = 0.25 \times N$. Then select a subset of the design rules to enforce where the ultimate goal is to include all design rules listed above with the exception of balance and $\pm 45^\circ$ rule of which one or the other is selected but not both.

In [14]:
# Generating Random Problem Instances
def generate_problem(N, dist):
    return np.array([N*dist[0], N*dist[1], N*dist[2], N*dist[3]]).astype(int)

In [15]:
dists = [[.25, .25, .25, .25], # percentage of 0, 45, -45, 90
         [.3, .3, .3, .1],
         [.4, .25, .25, .1],
         [.5, .2, .2, .1],
         [.2, .3, .3, .2],
         [.3, .2, .2, .3]]

In [17]:
dist = dists[0]
N_targ = 20
n_targ = generate_problem(N, dist)
print(n_targ)

[5 5 5 5]


## Branch for Problem Version 1

The next step in specifying a complete challenge problem description is to choose a subset of the design rules above to incorporate into the optimization. Consider using the rules **symmetry**, **damage tolerance**, **contiguity**, $\mathbf{\pm 45^\circ}$ **rule**, and **disorientation**. Symmetry is easy to take care of by changing the size of design vector $N \to N/2$; the cost function becomes $C (\boldsymbol \theta) = \sum_{j=1}^4 \left|2 \sum_{i=1}^{N/2} \delta_{\boldsymbol \theta_i, \theta_j} - n_j \right|$. 

Consider the oracle $\textbf{branch}$ in this case. $\textbf{branch}$ consumes a partially filled out design vector $\boldsymbol \theta^{(i)} = (\boldsymbol \theta_1, ..., \boldsymbol \theta_i, 0, 0, ..., 0)$. For this selection of design rules, we deduce that the action of $\textbf{branch}$ must be as follows

$\textbf{branch}(\boldsymbol \theta_i = 0^\circ) =  0^\circ \text{ if } \boldsymbol \theta_{i-1} \not = 0^\circ,  \begin{cases} 45^\circ & \text{if last 45 was} -45^\circ \\ -45^\circ & \text{if last 45 was} + 45^\circ \end{cases}$

$\textbf{branch}(\boldsymbol \theta_i = 90^\circ) =  90^\circ \text{ if } \boldsymbol \theta_{i-1} \not = 90^\circ, \begin{cases} 45^\circ & \text{if last 45 was} -45^\circ \\ -45^\circ & \text{if last 45 was} + 45^\circ \end{cases} $

$\textbf{branch}(\boldsymbol \theta_i = \pm 45^\circ) =  0^\circ, 90^\circ $

## Example Classical Brute Force Solution to Problem Version 1

In [18]:
def branch(theta, flag):
    """
        1 := 0 deg
        2 := 45 deg
        3 := -45 deg
        4 := 90 deg
        
        flag == 0: last 45 was +45
        flag == 1: last 45 was -45
    """
    children = []
    
    if theta[-1] == 2 or theta[-1] == 3: # if last angle was 45 or -45
        children.append(1) # append 0
        children.append(4) # append 90
    
    elif theta[-1] == 1 or theta[-1] == 4:
        if flag:
            children.append(2) # last 45 was -45 so append +45
        else:
            children.append(3) # last 45 was +45 so append -45
        
        flag = np.mod(flag+1, 2) # update flag
        
        if len(theta) > 2 and theta[-2] != theta[-1]: # If the last 2 angles are not the same
            children.append(theta[-1]) # append a repeated angle (0 or 90)
    
    return children, flag

In [19]:
def greedy_algorithm(N, n):
    """
        This algorithm is greedy and always selects the child that reduces the cost by the greatest amount
        If both children yield the same cost, choose one over the other randomly with probability 0.5
    """
    
    theta, flag = [2], 0 # Begin with +45 degree angle
    
    for i in range(1, N):
        children, flag = branch(theta, flag)
        
        if len(children) == 1: # If only one possible choice of an angle, choose it
            theta.append(children[0])
        
        else: # If there are two possible angle choices, compare them and choose one
            theta1 = theta.copy()
            theta2 = theta.copy()
        
            theta1.append(children[0])
            theta2.append(children[1])
            
            cost1 = cost(theta1, n)
            cost2 = cost(theta2, n)
            
            if cost1 < cost2: # choose angle children[0] for a lower cost
                theta = theta1
            
            elif cost1 > cost2: # choose angle children[1] for a lower cost
                theta = theta2
            
            else: # costs are equal, choose one of the two angles uniformly at random
                coin_flip = np.random.randint(0,2)
                
                if coin_flip == 0:
                    theta = theta1
                else:
                    theta = theta2
                    
    return theta

In [26]:
"""
    In order to achieve a cost of 0, I had to run this algorithm a few times
    
    The computational cost of this approach should be the expected value of the number of times
    I need to run the algorithm times the computational cost of each run of the algorithm:
    
                                    E[Runs] * (Algorithm Cost)
"""

theta_opt = greedy_algorithm(N_targ, n_targ)
print(theta_opt, cost(theta_opt, n_targ))

[2, 1, 3, 1, 2, 1, 3, 1, 2, 4, 3, 4, 2, 1, 3, 4, 2, 4, 3, 4] 0


## Realistic Target Problem Sizes

$40 \leq N \leq 160$

$D \leq 200$

## Idea for Quantum Implementation

We could imagine storing the design vector in a quantum state of $N$ qubits, $|\boldsymbol \theta^{(i)} \rangle_{N} = |\boldsymbol \theta_1, ..., \boldsymbol \theta_i, 0, ..., 0\rangle_{N}$ where each $\boldsymbol \theta_j \in \{0,1,2,3\}$ and therefore $|\boldsymbol \theta_j\rangle_2 = | \theta^1_j \theta^2_j \rangle_2$. 

We could also imagine adding an ancilla flag qubit that keeps track of the sign of the last 45 degree angle we saw. Then the algorithm's design vector state will look like

$|\psi_i\rangle_{N+1} = |\boldsymbol \theta^{(i)} \rangle_{N} \otimes |x\rangle_1$

where $x=0$ corresponds to a prior $+45^\circ$ angle and likewise $x=1$ corresponds to a prior $-45^\circ$ angle. Note that 
$|\psi_1 \rangle_{N+1} = |\boldsymbol \theta^{(1)} = \pm 45^\circ, 0,...,0 \rangle_N |x\rangle_1$ due to the damage tolerance design rule.

## Notes from the Boeing SME:

- I think overall the description given here for generating random problem instances and the associated rules needed makes good sense. If you wanted to extend it, I think the complexity of the random instances could be further expanded by using a randomized set of initial ply percentages for the given problem instance (maybe from a given list of common percentages), and possibly randomizing the parameters for some of the various design rules (such as randomizing the max permitted numbers of each angle for the Contiguity rules, or randomizing +45 or -45 is used for the damage tolerance rule, etc.). With these extensions, and with a sufficient problem size for both $N$ and $D$, this random instance approach would capture a very accurate representation of the complexity for actual industrial problems. The randomized instances should generally always include at least one of the Contiguity, +/-45, and Disorientation (preferably all three), as without any of these three rules the problem is often purely local (e.g., just consists of checking immediate neighbors of each ply only) and boils down the complexity of the real problem a whole lot.

- In general, we’re interested in both formulations that include the absent ply, and also in cases where it is not included. They differ in application, in what is being evaluated/optimized, and in what rules can be handled with their inclusion/exclusion, as to whether we decide to include it or not; one is not necessarily better than the other, but one may be more appropriate in certain situations based on what is desired to be optimized and what rules are needed to be included.
 
- It’s generally easier to both implement and to formulate the objective/constraint equations when we don’t include the blank ply, so sometimes it’s advantageous in particular cases to drop it from the ply set and focus just on formulating the problem without it. But the machines we use to manufacture the composites essentially must have a representation of a blank ply available when we provide instructions. It’s also sometimes not possible to determine manufacturability metrics without blanks, so if the cost function is aiming at manufacturability, we usually have to include them. Also, the evaluation of rules differs based on the inclusion of the blank ply – some rules get harder to evaluate when you include blanks, but some other rule formulations get a lot easier. So, it sometimes depends on which rules are needing to be included as to whether we use blanks or not, if it makes the mathematical implementation easier for the rules of interest.
 
- We’ve occasionally tried postprocessing a solution without blanks to add in the blanks afterward, but that generally doesn’t work very well; it’s a separate very difficult (and sometimes ill-posed) problem to find the best place to add in blanks, and you’ll still have essentially the same complexity in the problem either way, so you’re not avoiding or circumventing the issue by removing them from the initial problem formulation. There’s also a possibility of determining an optimal solution to the initial (non-blank) solution, but then finding a sub-optimal blank placement, which results in an overall sub-optimal solution to the full problem. We tend to opt to just include it, if it’s needed, to avoid these issues.

- There are many, many possible cost functions that could be selected here, and its selection is generally extremely design- and case-dependent. Some are motivated by physical requirements for the resulting composite. For example, we could choose a cost function that minimizes the torsion, where the torsion is a function of the chosen ply layers (many publications explore cases like this). In this particular instance, the cost function is motivated from a manufacturing standpoint, where the number of plies needed of each layer type have been pre-determined to meet some load requirement, and the cost function is driving a selection of layers that matches this requirement as best as possible

- The symmetry rule is actually a physically motivated rule; laminates with symmetric structure tend to withstand certain bending and twisting modes better. The computational performance benefit of solving a smaller problem size is actually incidental.

- It sounds from my reading here like we are trying to enforce the balance and min percent rules by applying requirements to the assigned input ply gauges, $n_j$. Please note that this generally must also be constrained on the actual solution vector as well within the algorithm; it is actually very possible to get a good solution (low cost function evaluation) that does not meet these rules, hence they cannot generally be handled only by constraining the input gauges, and must typically be evaluated in the branch function to help eliminate nonfeasible solutions.

- The +/-45 Rule is a constraint on the appearance of alternating plies to ensure that their total throughout the stack doesn't cause issues, whereas the Disorientation is a more local issue between neighboring physical plies, regardless of being +/-45 or not. As an example, a bad stack of [+45/0/0/+45] is weeded out by the +/-45 rule, but not by the Disorientation Rule, as it satisfies that one. Alternately the bad stack [+45/-45] satisfies the +/-45 Rule, but is weeded out by the Disorientation rule (since they have >45 degrees between them). If the absence ply $\emptyset$ is not included, then the Disorientation rule is purely local, and not as challenging of a constraint. If the absence ply $\emptyset$ is included, then this rule is effectively non-local and is harder to satisfy.

- The +/-45 Rule is actually an overlap and a re-statement of the Balance rule, and usually only one of either the +/-45 or the Balance rule is selected, since both are expressions of constraining the +/-45s. The difference is that the Balance rule only constrains by total number, whereas the +/-45 Rule constrains additionally the relative positioning in that they must flip sign per appearance, so is a stronger constraint.

- Note that the one exception in the +/-45 rule is at the midplane (the middle two sequences in even stacks), since otherwise symmetry will not be achieved. So at the midplane, you might see a [-45/-45], and this would not be considered a rule violation at this one place.

- Note that -45 degrees is considered only 45 degrees away from 90. This isn't numerically clear at all, but is pretty clear if you see the geometry of the angles on a fiber layer.

- What you're saying here about the branch function rejecting nodes that violate the Disorientation rule is true, and the description using $N_i$ and $N_{i+1}$ here is true if there are no empty plies in your solution vector. However, the nodes $N_i$ and $N_{i+1}$ must be *physical* layers, ignoring all/any absent/empty layers between them. This means that if there are absent plies, $\emptyset$, in your solution vector $\theta$, then you must evaluate $N_i$ and $N_{i+1}$ with any empty layers between them removed. For example, the solution vector [0/ * / * /90] (where * is the empty ply) still violates the Disorientation rule, so the branch function must recognize the Disorientation violation between nodes N_0 and N_3 here. This makes the problem more non-local and difficult when absent plies are included in the ply set, and makes the problem more challenging to keep track of and evaluate. However, we still do this within the branch function, we just need more tree bookkeeping to know where the last actual physical layers are in the solution vector to the current candidate nodes.

## References

[1] R. J. Thompson and A. W. Blom-schieber, SUBLAMINATE LIBRARY GENERATION FOR OPTIMIZATION OF MULTI-PANEL COMPOSITE PARTS, 20170228494 (2017).

[2] G. Ntourmas, F. Glock, F. Daoud, G. Schuhmacher, D. Chronopoulos, and E. Özcan, Mixed Integer Linear Programming Formulations of the Stacking Sequence and Blending Optimisation of Composite Structures, Composite Structures 264, 113660 (2021).